In [1]:
import getpass
import os
from typing import List, Dict, Optional
import json
import requests
from pydantic import BaseModel, Field

In [2]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

index_name = "augmentedllm"
_set_env("OPENAI_API_KEY")
_set_env("PINECONE_API_KEY")

OPENAI_API_KEY:  ········
PINECONE_API_KEY:  ········


In [3]:
# Helper class for running dialogue tests
class DialogueTest:
    """Simple framework for testing LLM dialogue capabilities."""
    
    def __init__(self, agent):
        """Initialize with an LLM agent to test."""
        self.agent = agent
    
    def chat(self, message: str) -> str:
        """Send a message and get the response."""
        print(f"\nUser: {message}")
        response = self.agent.process_message(message)
        print(f"Assistant: {response}")
        return response

In [4]:
# Part 1: Basic LLM Implementation
print("\n=== Part 1: Basic LLM ===")
print("Simple LLM that can chat but doesn't remember context")


=== Part 1: Basic LLM ===
Simple LLM that can chat but doesn't remember context


In [5]:
class BasicLLM:
    """Basic LLM with just chat capability."""
    
    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        self.api_key = api_key
        self.model = model
        
    def process_message(self, message: str) -> str:
        """Process a single message."""
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "messages": [
                        {
                            "role": "system",
                            "content": "You are a helpful AI assistant."
                        },
                        {
                            "role": "user",
                            "content": message
                        }
                    ]
                }
            )
            response.raise_for_status()
            return response.json()['choices'][0]['message']['content']
            
        except Exception as e:
            return f"Error: {str(e)}"

In [7]:
# Test Basic LLM
basic_llm = BasicLLM(os.environ["OPENAI_API_KEY"])
basic_test = DialogueTest(basic_llm)

print("\nTesting Basic LLM:")
basic_test.chat("Hi! My name is Viacheslav.")
basic_test.chat("Do you remember my name?")


Testing Basic LLM:

User: Hi! My name is Alice.
Assistant: Hi Alice! How can I assist you today?

User: Do you remember my name?
Assistant: I don’t have the ability to remember personal details or past interactions, including names. How can I assist you today?


'I don’t have the ability to remember personal details or past interactions, including names. How can I assist you today?'

In [8]:
# Part 2: LLM with Memory
print("\n=== Part 2: LLM with Memory ===")
print("Now our LLM can remember conversation context")


=== Part 2: LLM with Memory ===
Now our LLM can remember conversation context


In [11]:
class Message(BaseModel):
    """Single message in conversation history."""
    role: str
    content: str

class MemoryLLM(BasicLLM):
    """LLM with conversation memory."""
    
    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        super().__init__(api_key, model)
        self.history: List[Message] = []
        
    def process_message(self, message: str) -> str:
        """Process message with conversation history."""
        try:
            # Prepare messages including history
            messages = [{"role": "system", "content": "You are a helpful marketing assistant. Answer just if you sure about correct info 100%. If not say you not sure."}]
            messages.extend([msg.dict() for msg in self.history])
            messages.append({"role": "user", "content": message})
            
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "messages": messages
                }
            )
            response.raise_for_status()
            
            # Get response and update history
            assistant_message = response.json()['choices'][0]['message']['content']
            self.history.append(Message(role="user", content=message))
            self.history.append(Message(role="assistant", content=assistant_message))
            
            return assistant_message
            
        except Exception as e:
            return f"Error: {str(e)}"

In [12]:
 # Test 2: LLM with Memory
print("\n=== Testing LLM with Memory ===")
memory_llm = MemoryLLM(os.environ["OPENAI_API_KEY"])
memory_test = DialogueTest(memory_llm)
memory_test.chat("Hi! My name is Viacheslav.")
memory_test.chat("Do you remember my name?") 
memory_test.chat("What you khow about The Global Sourcing RFI Company?") 


=== Testing LLM with Memory ===

User: Hi! My name is Viacheslav.
Assistant: Hello Viacheslav! How can I assist you today?

User: Do you remember my name?
Assistant: Yes, your name is Viacheslav. How can I help you today?

User: What you khow about The Global Sourcing RFI Company?
Assistant: I'm not sure about specific current details regarding The Global Sourcing RFI Company, as my training data only goes up to October 2023, and I don't have direct knowledge about every company. If you need information about their services, reputation, or other specific inquiries, I recommend checking their official website or recent news articles for the most accurate and up-to-date information.


"I'm not sure about specific current details regarding The Global Sourcing RFI Company, as my training data only goes up to October 2023, and I don't have direct knowledge about every company. If you need information about their services, reputation, or other specific inquiries, I recommend checking their official website or recent news articles for the most accurate and up-to-date information."

In [13]:
# Part 3a: Document Processing and Indexing
print("\n=== Part 3a: Document Processing and Indexing ===")
print("First step of RAG: Processing and storing documents")


=== Part 3a: Document Processing and Indexing ===
First step of RAG: Processing and storing documents


In [20]:
class DocumentProcessor:
    """Processes documents into chunks."""
    
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def create_chunks(self, text: str) -> List[Dict[str, str]]:
        """Split text into overlapping chunks."""
        if not text:
            raise ValueError("Empty text provided")
        if self.chunk_size <= self.chunk_overlap:
            raise ValueError("chunk_size must be greater than chunk_overlap")
        
        chunks = []
        start = 0
        
        while start < len(text):
            end = start + self.chunk_size
            chunk = text[start:end]
            
            chunks.append({
                'text': chunk,
                'metadata': {'start_char': start, 'end_char': end}
            })
            
            start += self.chunk_size - self.chunk_overlap
            
        print(f"\nCreated {len(chunks)} chunks")
        print(f"Sample chunk (first 50 chars): {chunks[0]['text'][:50]}...")
        return chunks

In [21]:
class EmbeddingGenerator:
    """Handles text embedding generation."""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.api_url = "https://api.openai.com/v1/embeddings"
        
    def create_embedding(self, text: str) -> List[float]:
        """Generate embedding for text."""
        if not text.strip():
            raise ValueError("Empty text provided")
            
        response = requests.post(
            self.api_url,
            headers={
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": "text-embedding-3-small",
                "input": text
            }
        )
        response.raise_for_status()
        embedding = response.json()['data'][0]['embedding']
        print(f"Generated embedding, sample of first 3 dimensions: {embedding[:3]}")
        return embedding

In [22]:
class VectorStore:
    """Manages vector storage operations."""

    def __init__(self, api_key: str, index_name: str):
        self.api_key = api_key
        self.index_name = index_name
        self.headers = {
            "Api-Key": self.api_key,
            "Content-Type": "application/json",
            "X-Pinecone-API-Version": "2024-07"
        }
        self.index_host_cache = {}
        
    def describe_index(self) -> str:
        """Get or retrieve cached index host."""
        if self.index_name in self.index_host_cache:
            return self.index_host_cache[self.index_name]
            
        url = f"https://api.pinecone.io/indexes/{self.index_name}"
        response = requests.get(url, headers=self.headers)
        response.raise_for_status()
        
        host = response.json()["host"]
        self.index_host_cache[self.index_name] = host
        print(f"\nConnected to Pinecone host: {host}")
        return host
        
    def query_vectors(self, 
                      query_vector: List[float], 
                      top_k: int = 3, 
                      namespace: str = "") -> List[Dict]:
        """Query for most similar vectors."""
        host = self.describe_index()
        url = f"https://{host}/query"
        data = {
            "vector": query_vector,
            "topK": top_k,
            "namespace": namespace,
            "includeMetadata": True
        }
        
        print(f"\nQuerying Pinecone for top {top_k} matches")
        response = requests.post(url, headers=self.headers, json=data)
        response.raise_for_status()
        
        matches = response.json()['matches']
        return matches
        
    def store_vectors(self, vectors: List[Dict], namespace: str = "") -> Dict:
        """Store vectors in Pinecone."""
        host = self.describe_index()
        url = f"https://{host}/vectors/upsert"
        data = {
            "vectors": vectors,
            "namespace": namespace
        }
        
        print(f"\nUploading {len(vectors)} vectors to Pinecone")
        print(f"Sample vector metadata: {vectors[0]['metadata']}")
        
        response = requests.post(url, headers=self.headers, json=data)    
        response.raise_for_status()
        result = response.json()
        print(f"Pinecone response: {result}")
        return result

In [23]:
def process_and_store_document(text: str, document_id: str, openai_key: str, pinecone_key: str, index_name: str) -> bool:
    """Process document and store in vector database."""
    try:
        # Initialize components
        processor = DocumentProcessor()
        embedding_gen = EmbeddingGenerator(openai_key)
        vector_store = VectorStore(pinecone_key, index_name)
        
        # Create chunks
        chunks = processor.create_chunks(text)
        
        # Generate embeddings and prepare vectors
        vectors = []
        for i, chunk in enumerate(chunks):
            embedding = embedding_gen.create_embedding(chunk['text'])
            if embedding:
                vectors.append({
                    'id': f"{document_id}-{i}",
                    'values': embedding,
                    'metadata': {
                        'text': chunk['text'],
                        'document_id': document_id,
                        **chunk['metadata']
                    }
                })
        
        # Store vectors
        if vectors:
            vector_store.store_vectors(vectors)
            return True
        return False
            
    except Exception as e:
        print(f"Error processing document: {e}")
        return False

In [18]:
# Part 3b: LLM with Memory and RAG
print("\n=== Part 3b: LLM with Memory and RAG ===")
print("Adding Retrieval Augmented Generation capability")


=== Part 3b: LLM with Memory and RAG ===
Adding Retrieval Augmented Generation capability


In [27]:
class RagLLM(MemoryLLM):
    """LLM with conversation memory and RAG capability."""
    
    def __init__(self, openai_key: str, pinecone_key: str, index_name: str, model: str = "gpt-4o-mini"):
        super().__init__(openai_key, model)
        self.pinecone_key = pinecone_key
        self.index_name = index_name
        self.embedding_generator = EmbeddingGenerator(openai_key)
        self.vector_store = VectorStore(pinecone_key, index_name)
        
    def _get_relevant_chunks(self, text: str, top_k: int = 3) -> List[str]:
        """Get relevant text chunks using vector search."""
        try:
            # Get query embedding
            query_vector = self.embedding_generator.create_embedding(text)
            
            # Query vector store
            matches = self.vector_store.query_vectors(
                query_vector=query_vector,
                top_k=top_k
            )
            
            # Extract texts from metadata
            return [match['metadata'].get('text', '') for match in matches]
            
        except Exception as e:
            print(f"Error getting relevant chunks: {e}")
            return []
            
    def process_message(self, message: str) -> str:
        """Process message with conversation history and relevant context."""
        try:
            # Get relevant context
            context_chunks = self._get_relevant_chunks(message)
            context = "\n".join(context_chunks)
            print(f"\nFound {len(context_chunks)} relevant chunks")
            if context_chunks:
                print(f"Sample context (first 50 chars): {context_chunks[0][:50]}...")
            
            # Prepare messages
            messages = [{
                "role": "system",
                "content": f"""You are a helpful marketing assistant. Answer just if you sure about correct info 100%. If not say you not sure. 
                Use this context when relevant:
                {context}
                """
            }]
            messages.extend([msg.dict() for msg in self.history])
            messages.append({"role": "user", "content": message})
            
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "messages": messages
                }
            )
            response.raise_for_status()
            
            # Get response and update history
            assistant_message = response.json()['choices'][0]['message']['content']
            self.history.append(Message(role="user", content=message))
            self.history.append(Message(role="assistant", content=assistant_message))
            
            return assistant_message
            
        except Exception as e:
            return f"Error: {str(e)}"

In [26]:
print("\n=== Testing Document Processing ===")
with open("your doc.txt", "r") as f:
    text = f.read()
    success = process_and_store_document(
        text=text,
        document_id="yourDoc",
        openai_key=os.environ["OPENAI_API_KEY"],
        pinecone_key=os.environ["PINECONE_API_KEY"],
        index_name="augmentedllm"
    )
    print(f"\nDocument processing {'succeeded' if success else 'failed'}")


=== Testing Document Processing ===

Created 13 chunks
Sample chunk (first 50 chars): ﻿Global Sourcing RFI Workflow


Global Sourcing RF...
Generated embedding, sample of first 3 dimensions: [-0.009754701, 0.023795739, 0.010035283]
Generated embedding, sample of first 3 dimensions: [-0.04238617, 0.009184542, 0.010165098]
Generated embedding, sample of first 3 dimensions: [-0.03158922, 0.03678677, 0.020493945]
Generated embedding, sample of first 3 dimensions: [-0.0501792, 0.040126923, 0.057081576]
Generated embedding, sample of first 3 dimensions: [-0.04344335, 0.011441238, 0.02943367]
Generated embedding, sample of first 3 dimensions: [-0.021920724, 0.040596727, 0.025479855]
Generated embedding, sample of first 3 dimensions: [-0.047885858, -0.004947266, 0.02059778]
Generated embedding, sample of first 3 dimensions: [-0.054924604, -0.008015131, 0.035972822]
Generated embedding, sample of first 3 dimensions: [-0.047217026, -0.02386938, 0.020438973]
Generated embedding, sample of first 

In [28]:
# Test 3b: LLM with Memory and RAG
print("\n=== Testing LLM with Memory and RAG ===")
rag_llm = RagLLM(os.environ["OPENAI_API_KEY"], os.environ["PINECONE_API_KEY"], "augmentedllm")
rag_test = DialogueTest(rag_llm)
rag_test.chat("Hi! My name is Viacheslav.")
rag_test.chat("Do you remember my name?")
rag_test.chat("What you khow about The Global Sourcing RFI Company?")
rag_test.chat("We spent $10,000 on a marketing campaign that generated $50,000 in revenue with a 10% margin. What was our ROMI?")


=== Testing LLM with Memory and RAG ===

User: Hi! My name is Viacheslav.
Generated embedding, sample of first 3 dimensions: [0.015347642, -0.020478658, -0.042047087]

Connected to Pinecone host: augmentedllm-cxx1dj3.svc.aped-4627-b74a.pinecone.io

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 50 chars): izing in supplier identification and matching. Vie...
Assistant: Hello, Viacheslav! How can I assist you today?

User: Do you remember my name?
Generated embedding, sample of first 3 dimensions: [0.036457118, -0.07861758, -0.049314216]

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 50 chars): e workflow will check for available contact inform...
Assistant: Yes, your name is Viacheslav. How can I assist you further?

User: What you khow about The Global Sourcing RFI Company?
Generated embedding, sample of first 3 dimensions: [0.0016378722, -0.012900318, 0.034605835]

Querying Pinecone for top 3 matches

Found 3 r

'To calculate the Return on Marketing Investment (ROMI), you can use the following formula:\n\n\\[ \\text{ROMI} = \\frac{\\text{Revenue} - \\text{Marketing Investment}}{\\text{Marketing Investment}} \\]\n\nIn this case:\n\n- Revenue = $50,000\n- Marketing Investment = $10,000\n\nFirst, calculate the profit generated:\n\n\\[ \\text{Profit} = \\text{Revenue} \\times \\text{Margin} = 50,000 \\times 0.10 = 5,000 \\]\n\nNow, plug in the values into the ROMI formula:\n\n\\[ \\text{ROMI} = \\frac{50,000 - 10,000}{10,000} = \\frac{40,000}{10,000} = 4 \\]\n\nSo, the ROMI is 4, or 400%. This means for every dollar spent on marketing, you generated 4 dollars in revenue.'

In [29]:
# Part 4: LLM with Tools
print("\n=== Part 4: LLM with Tools ===")
print("Adding marketing analytics tools")


=== Part 4: LLM with Tools ===
Adding marketing analytics tools


In [30]:
class MarketingTools:
    """Provides marketing analytics calculations."""
    
    @staticmethod
    def get_tool_definitions() -> List[Dict]:
        return [{
            "type": "function",
            "function": {
                "name": "calculate_romi",
                "description": "Calculate Return on Marketing Investment (ROMI)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "revenue": {
                            "type": "number",
                            "description": "Total revenue generated from marketing campaign"
                        },
                        "marketing_cost": {
                            "type": "number",
                            "description": "Total cost of marketing campaign"
                        },
                        "margin_percent": {
                            "type": "number",
                            "description": "Profit margin percentage (0-100)"
                        }
                    },
                    "required": ["revenue", "marketing_cost", "margin_percent"]
                }
            }
        }]
    
    @staticmethod
    def calculate_romi(revenue: float, marketing_cost: float, margin_percent: float) -> float:
        """
        Calculate ROMI using the formula: ((Revenue * Margin%) - Marketing Cost) / Marketing Cost * 100
        Returns percentage value
        """
        if marketing_cost <= 0:
            raise ValueError("Marketing cost must be greater than zero")
        if not (0 <= margin_percent <= 100):
            raise ValueError("Margin percentage must be between 0 and 100")
            
        margin_multiplier = margin_percent / 100
        profit = revenue * margin_multiplier - marketing_cost
        romi = (profit / marketing_cost) * 100
        return round(romi, 2)

In [31]:
class ToolEnabledLLM(RagLLM):
    """LLM with memory, RAG and tool usage capability."""
    
    def __init__(self, 
                 openai_key: str, 
                 pinecone_key: str, 
                 index_name: str, 
                 model: str = "gpt-4o-mini"):
        super().__init__(openai_key, pinecone_key, index_name, model)
        self.tools = MarketingTools()
    
    def process_message(self, message: str) -> str:
        """Process message with tools, memory and RAG."""
        try:
            # Get relevant context
            context_chunks = self._get_relevant_chunks(message)
            context = "\n".join(context_chunks)
            print(f"\nFound {len(context_chunks)} relevant chunks")
            if context_chunks:
                print(f"Sample context (first 100 chars): {context_chunks[0][:100]}...")
            
            # Prepare messages
            messages = [{
                "role": "system",
                "content": f"""You are a marketing analytics assistant. 
                Use this context when relevant:
                {context}
                
                When asked about marketing metrics, use the calculate_romi tool.
                ROMI (Return on Marketing Investment) shows the profitability of marketing spending.
                """
            }]
            messages.extend([msg.dict() for msg in self.history])
            messages.append({"role": "user", "content": message})
            
            # Make API request with tools
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "messages": messages,
                    "tools": self.tools.get_tool_definitions()
                }
            )
            response.raise_for_status()
            
            # Process response and tool calls
            response_data = response.json()
            assistant_message = response_data['choices'][0]['message']
            
            if tool_calls := assistant_message.get('tool_calls'):
                print("\nProcessing tool calls...")
                tool_results = self._handle_tool_calls(tool_calls)
                
                # Add tool results to conversation
                messages.extend([
                    {
                        "role": "assistant",
                        "content": assistant_message.get('content'),
                        "tool_calls": tool_calls
                    },
                    {
                        "role": "tool",
                        "content": json.dumps(tool_results),
                        "tool_call_id": tool_calls[0]['id']
                    }
                ])
                
                # Get final response with tool results
                final_response = requests.post(
                    "https://api.openai.com/v1/chat/completions",
                    headers={
                        "Authorization": f"Bearer {self.api_key}",
                        "Content-Type": "application/json"
                    },
                    json={
                        "model": self.model,
                        "messages": messages
                    }
                )
                final_response.raise_for_status()
                assistant_message = final_response.json()['choices'][0]['message']
            
            # Update conversation history
            self.history.append(Message(role="user", content=message))
            self.history.append(Message(role="assistant", content=assistant_message['content']))
            
            return assistant_message['content']
            
        except Exception as e:
            return f"Error: {str(e)}"
            
    def _handle_tool_calls(self, tool_calls: List[Dict]) -> Dict:
        """Process tool calls and return results."""
        results = {}
        
        for call in tool_calls:
            if call['function']['name'] == 'calculate_romi':
                try:
                    args = json.loads(call['function']['arguments'])
                    result = self.tools.calculate_romi(
                        revenue=args['revenue'],
                        marketing_cost=args['marketing_cost'],
                        margin_percent=args['margin_percent']
                    )
                    print(f"Calculated ROMI: {result}%")
                    results[call['id']] = result
                except Exception as e:
                    results[call['id']] = f"Error calculating ROMI: {str(e)}"
                    
        return results

In [33]:
# Test 4: LLM with Tools
print("\n=== Testing Tool-enabled LLM ===")
tool_llm = ToolEnabledLLM(os.environ["OPENAI_API_KEY"], os.environ["PINECONE_API_KEY"], "augmentedllm")
tool_test = DialogueTest(tool_llm)
tool_test.chat("Hi! My name is Slava?")
tool_test.chat("Do you remember my name?")
tool_test.chat("How will we translate Vietnamese Data into Vietnamese?")
tool_test.chat("We spent $10,000 on a marketing campaign that generated $50,000 in revenue with a 10% margin. What was our ROMI?")
tool_test.chat("How about a campaign where we spent $200,000 and got $4,000,000 in revenue with 50% margin?")


=== Testing Tool-enabled LLM ===

User: Hi! My name is Slava?
Generated embedding, sample of first 3 dimensions: [0.008466213, -0.008110435, -0.04214]

Connected to Pinecone host: augmentedllm-cxx1dj3.svc.aped-4627-b74a.pinecone.io

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 100 chars): izing in supplier identification and matching. Vientam Direct Sourcing is designed to connect medium...
Assistant: Hello Slava! How can I assist you today?

User: Do you remember my name?
Generated embedding, sample of first 3 dimensions: [0.036457118, -0.07861758, -0.049314216]

Querying Pinecone for top 3 matches

Found 3 relevant chunks
Sample context (first 100 chars): e workflow will check for available contact information: if present, an email will be sent to the cl...
Assistant: Yes, I remember your name, Slava! How can I help you today?

User: How will we translate Vietnamese Data into Vietnamese?
Generated embedding, sample of first 3 dimensions: [-0.000

'For the campaign where you spent $200,000 and generated $4,000,000 in revenue with a 50% margin, your ROMI is 900%. This indicates a highly profitable marketing investment. If you need further analysis or assistance, feel free to ask!'